In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('./train.csv')
test=pd.read_csv('./test.csv')
df.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,Music video by Luke Bryan performing Out Of No...
1,hNsd-m30m2E,24.0,thanks for watching!!! happy holidays everyone...
2,yLdav-8af6g,1.0,Marcus and Adam build their own custom lightsa...
3,Ii3Rr4-Pxow,26.0,Make sure you subscribe to my channel and hit ...
4,515lLZS082U,24.0,It's a riff-off pitches! Get Tix: http://unvrs...


In [3]:
test.head()

,video_id,description
0,2kyS6SvSYSE,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,_6lGaYh71g4,Balmuda: the Toaster is a Japanese invention t...
2,X_tYrnv_o6A,These are the molecular machines inside your b...
3,BEcs3TL8DII,LeBron James is 'obviously' the NBA MVP this s...
4,26qOMpjicGs,Lol - Subscribe to my new channel that will co...


In [4]:
df.shape

(3471, 3)

In [5]:
df = df[pd.notnull(df['description'])]

In [6]:
df.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,Music video by Luke Bryan performing Out Of No...
1,hNsd-m30m2E,24.0,thanks for watching!!! happy holidays everyone...
2,yLdav-8af6g,1.0,Marcus and Adam build their own custom lightsa...
3,Ii3Rr4-Pxow,26.0,Make sure you subscribe to my channel and hit ...
4,515lLZS082U,24.0,It's a riff-off pitches! Get Tix: http://unvrs...


In [7]:

df.shape


(3471, 3)

In [8]:
X=df['description']
y=df['category_id']

In [9]:
X.shape

(3471,)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.description).toarray()
labels = df.category_id
features.shape

(3471, 170139)

In [11]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))


NameError: name 'category_to_id' is not defined

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['description'], df['category_id'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [13]:
print(clf.predict(count_vect.transform([X_test[2831]])))

[24.]


In [14]:
X_test

196     Build your website for 10% off over at http://...
2213    Source: https://www.youtube.com/watch?v=sAooq7...
1784    iMac Pro. Turns out it's not a trap. It's pret...
1129    Pit Bull Brings Her Puppies To Her Foster Mom ...
2515    Download & Stream “Change (feat. James Taylor)...
                              ...                        
1462    Soooo that happened...\n\nFLIPPER- https://www...
2043    Learn from over 17,000 courses with Skillshare...
981     Charles Barkley and Shaquille O'Neal find it f...
3313    Bon Appétit Test Kitchen Manager Brad Leone is...
2831    This is a demonstration video of crimping the ...
Name: description, Length: 868, dtype: object

In [15]:
y_test[2831]

28.0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
#     RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
#     LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


'3.8.3 (default, Jul  2 2020, 16:21:59) \n[GCC 7.3.0]'

In [2]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

df = pd.read_csv('train.csv')
# df = df[pd.notnull(df['tags'])]
print(df.head(10))
print(df['description'].apply(lambda x: len(x.split(' '))).sum())

      video_id  category_id                                        description
0  7NcVNt7zQJM         10.0  Music video by Luke Bryan performing Out Of No...
1  hNsd-m30m2E         24.0  thanks for watching!!! happy holidays everyone...
2  yLdav-8af6g          1.0  Marcus and Adam build their own custom lightsa...
3  Ii3Rr4-Pxow         26.0  Make sure you subscribe to my channel and hit ...
4  515lLZS082U         24.0  It's a riff-off pitches! Get Tix: http://unvrs...
5  3fArI2PZBUo         22.0  40cm of fresh snow meets a 4 month old corgi p...
6  cyhU06cXfeU         26.0  Are you obsessed with Christmas? Here are 10 w...
7  F3eSaEHSCu0         24.0  🎬 LIVE | EN DIRECTO | EN DIRECTE\n💪 Warm-up. C...
8  ZrhKHytNLqY         25.0  During the Grammys, singer and actress Janelle...
9  IGScS6YgzBY         23.0  Dish Nation is a FOX syndicated TV show that d...
394948


In [3]:
def print_plot(index):
    example = df[df.index == index][['description', 'category_id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

print_plot(10)

Chloe x Halle's debut album 'The Kids Are Alright' featuring Warrior comes out on March 23rd! Pre-order now to get 5 tracks instantly: http://smarturl.it/TKAA?iqid=pseudo\n\nAmazon: http://smarturl.it/TKAA/az \nGoogle Play: http://smarturl.it/TKAA/googleplay \niTunes: http://smarturl.it/TKAA/itunes \nSpotify Presave: http://smarturl.it/TKAA/spotifypresave \n \nListen now: \nApple Music: http://smarturl.it/TKAA/applemusic \nSpotify: http://smarturl.it/TKAA/spotify\nTidal: http://smarturl.it/TKAA/tidal \n\n------\n\nFollow Chloe x Halle: \nhttp://chloexhalle.com\nhttp://twitter.com/chloexhalle\nhttp://facebook.com/chloexhalle\nhttp://instagram.com/chloexhalle
Tag: 10.0


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/himanshu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,.;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
main=set(['http','https','nhttp','n','com','www'])
STOPWORDS=STOPWORDS|main

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(' ', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['description'] = df['description'].apply(clean_text)
print_plot(1)

/home/himanshu/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:414: MarkupResemblesLocatorWarning: "https://nyti.ms/2q1IjTV\nhttps://wapo.st/2GvQRgX?tid=ss_tw&utm_term=.6e116c3b2925\nhttp://www.sacbee.com/news/business/article207909079.html\nhttps://fxn.ws/2q3Fj9B" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/himanshu/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:414: MarkupResemblesLocatorWarning: "http://www.monsterstore.com\nhttps://www.facebook.com/MonsterProducts/\nhttps://twitter.com/monsterproducts\nhttps://www.youtube.com/monsterproducts\nhttps://www.instagram.com/monsterproducts/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/himanshu/anaco

thanks watching happy holidays everyone links socials nsocials maddieziegler maddiestyle nmaddie style website maddiestyle nthis holiday sweater maddiestyle holiday sweater nxoxoooo
Tag: 24.0


/home/himanshu/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:414: MarkupResemblesLocatorWarning: "https://www.facebook.com/thekeyboardcat\nhttp://www.charlieschmidt.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [7]:
txt="Революция 1917 года и последовавшие за ней политические, социальные и экономические изменения принесли новую терминологию и значительно увеличили количество международных слов в русском словаре. Реформа орфографии 1918 года придала письменному русскому языку современный вид. Грамотность стала почти всеобщей. Достижения в военной, научной, технологической областях. области искусства, а также освоение космоса придали современной России всемирный престиж, который шел вместе с ее имиджем сверхдержавы."

In [9]:
txt=clean_text(txt)

In [10]:
txt

'1917 1918'

In [5]:
df['description'].apply(lambda x: len(x.split(' '))).sum()

354885

In [6]:
df.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,music video luke bryan performing nowhere girl...
1,hNsd-m30m2E,24.0,thanks watching happy holidays everyone links ...
2,yLdav-8af6g,1.0,marcus adam build custom lightsabers always tw...
3,Ii3Rr4-Pxow,26.0,make sure subscribe channel hit notification b...
4,515lLZS082U,24.0,riff pitches get tix unvrs al pp3tix n#pitchpe...


In [7]:

#Creating the features (tf-idf weights) for the processed text

texts = df['description'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X1 = tfidf_vectorizer.fit_transform(texts) #features
y = df['category_id'].values #target

print (X1.shape)
print(y.shape)

(3471, 84534)
(3471,)


In [26]:
# X = df.description
lsa = TruncatedSVD(n_components=1000, n_iter=10, random_state=3)
X = lsa.fit_transform(X1)
print(X.shape)
# y = df.category_id
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

(3471, 1000)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [28]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=10, tol=None)
sgd.fit(X_train, y_train)

# %%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8426103646833013
              precision    recall  f1-score   support

         1.0       0.87      0.75      0.80        71
         2.0       1.00      0.77      0.87        13
        10.0       0.77      0.97      0.86       151
        15.0       1.00      0.82      0.90        22
        17.0       0.95      0.80      0.87        51
        19.0       1.00      0.67      0.80         3
        20.0       0.92      0.57      0.71        21
        22.0       0.93      0.78      0.84        80
        23.0       0.83      0.82      0.83        84
        24.0       0.80      0.86      0.83       243
        25.0       0.86      0.91      0.89        70
        26.0       0.92      0.86      0.89       112
        27.0       0.75      0.88      0.81        41
        28.0       0.90      0.76      0.82        79
        29.0       0.00      0.00      0.00         1
        43.0       0.00      0.00      0.00         0

    accuracy                           0.84      104

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
count=0
for i,x in enumerate(y_pred):
    if(x!=y_test[i]):
        print(x,y_test[i])
        count+=1
print("count: ",count)

28.0 26.0
10.0 23.0
10.0 23.0
10.0 23.0
25.0 28.0
24.0 28.0
1.0 24.0
22.0 24.0
10.0 22.0
10.0 1.0
25.0 22.0
26.0 24.0
24.0 27.0
24.0 28.0
23.0 20.0
26.0 10.0
10.0 22.0
10.0 1.0
10.0 24.0
26.0 28.0
10.0 25.0
10.0 24.0
24.0 1.0
23.0 26.0
23.0 27.0
24.0 17.0
10.0 24.0
1.0 10.0
23.0 24.0
10.0 28.0
24.0 27.0
24.0 26.0
24.0 23.0
28.0 1.0
10.0 28.0
24.0 1.0
24.0 20.0
1.0 27.0
10.0 28.0
10.0 25.0
10.0 24.0
24.0 26.0
27.0 17.0
10.0 15.0
24.0 1.0
10.0 22.0
43.0 28.0
24.0 26.0
24.0 26.0
24.0 28.0
24.0 28.0
26.0 22.0
24.0 2.0
22.0 23.0
27.0 26.0
1.0 20.0
24.0 1.0
25.0 24.0
10.0 17.0
10.0 27.0
24.0 1.0
26.0 24.0
23.0 17.0
24.0 25.0
1.0 24.0
27.0 26.0
1.0 24.0
10.0 24.0
24.0 1.0
27.0 25.0
24.0 17.0
10.0 22.0
25.0 22.0
26.0 28.0
24.0 28.0
17.0 24.0
10.0 26.0
20.0 22.0
28.0 24.0
10.0 26.0
10.0 24.0
24.0 1.0
23.0 24.0
1.0 26.0
26.0 23.0
25.0 1.0
10.0 24.0
10.0 26.0
10.0 1.0
23.0 17.0
27.0 24.0
24.0 20.0
24.0 23.0
10.0 22.0
24.0 22.0
23.0 24.0
24.0 1.0
24.0 26.0
24.0 22.0
10.0 24.0
24.0 1.0
24.0 1.0
29.

In [ ]:
df_test=pd.read_csv()

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

# %%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

In [9]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=10, tol=None)),
               ])
sgd.fit(X_train, y_train)

# %%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8330134357005758
              precision    recall  f1-score   support

         1.0       0.86      0.68      0.76        71
         2.0       1.00      0.62      0.76        13
        10.0       0.79      0.97      0.88       151
        15.0       0.95      0.82      0.88        22
        17.0       0.93      0.76      0.84        51
        19.0       0.50      0.33      0.40         3
        20.0       0.92      0.52      0.67        21
        22.0       0.94      0.78      0.85        80
        23.0       0.83      0.81      0.82        84
        24.0       0.78      0.86      0.82       243
        25.0       0.86      0.91      0.89        70
        26.0       0.83      0.88      0.85       112
        27.0       0.82      0.88      0.85        41
        28.0       0.89      0.75      0.81        79
        29.0       0.00      0.00      0.00         1
        43.0       0.00      0.00      0.00         0

    accuracy                           0.83      104

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/pyth

In [10]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

# %%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8416506717850288
              precision    recall  f1-score   support

         1.0       0.94      0.70      0.81        71
         2.0       1.00      0.69      0.82        13
        10.0       0.88      0.93      0.91       151
        15.0       1.00      0.82      0.90        22
        17.0       0.95      0.78      0.86        51
        19.0       0.67      0.67      0.67         3
        20.0       1.00      0.62      0.76        21
        22.0       0.84      0.79      0.81        80
        23.0       0.67      0.81      0.74        84
        24.0       0.76      0.89      0.82       243
        25.0       0.94      0.87      0.90        70
        26.0       0.93      0.88      0.90       112
        27.0       0.94      0.83      0.88        41
        28.0       0.85      0.80      0.82        79
        29.0       0.00      0.00      0.00         1
        43.0       0.00      0.00      0.00         0

    accuracy                           0.84      104

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-define

In [15]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf',RandomForestClassifier(random_state=42)),
               ])
logreg.fit(X_train, y_train)

# %%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8032629558541267
              precision    recall  f1-score   support

         1.0       0.98      0.70      0.82        71
         2.0       1.00      0.46      0.63        13
        10.0       0.79      0.95      0.86       151
        15.0       1.00      0.73      0.84        22
        17.0       0.94      0.65      0.77        51
        19.0       1.00      0.33      0.50         3
        20.0       0.92      0.52      0.67        21
        22.0       0.94      0.78      0.85        80
        23.0       0.57      0.81      0.67        84
        24.0       0.71      0.91      0.80       243
        25.0       0.96      0.73      0.83        70
        26.0       0.92      0.82      0.87       112
        27.0       0.91      0.76      0.83        41
        28.0       0.94      0.65      0.77        79
        29.0       0.00      0.00      0.00         1
        43.0       0.00      0.00      0.00         0

    accuracy                           0.80      104

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/pyth

In [16]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf',KNeighborsClassifier()),
               ])
logreg.fit(X_train, y_train)

# %%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.682341650671785
              precision    recall  f1-score   support

         1.0       0.67      0.68      0.67        71
         2.0       0.64      0.54      0.58        13
        10.0       0.77      0.86      0.81       151
        15.0       0.81      0.77      0.79        22
        17.0       0.77      0.78      0.78        51
        19.0       0.29      0.67      0.40         3
        20.0       0.67      0.48      0.56        21
        22.0       0.57      0.56      0.57        80
        23.0       0.52      0.76      0.62        84
        24.0       0.65      0.67      0.66       243
        25.0       0.70      0.63      0.66        70
        26.0       0.76      0.62      0.68       112
        27.0       0.80      0.78      0.79        41
        28.0       0.85      0.49      0.62        79
        29.0       0.00      0.00      0.00         1
        43.0       0.00      0.00      0.00         0

    accuracy                           0.68      1042

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/pyth

In [17]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(random_state=3)),
               ])
logreg.fit(X_train, y_train)

# %%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7293666026871402
              precision    recall  f1-score   support

         1.0       0.66      0.68      0.67        71
         2.0       0.47      0.54      0.50        13
        10.0       0.87      0.89      0.88       151
        15.0       0.73      0.73      0.73        22
        17.0       0.73      0.73      0.73        51
        19.0       0.25      0.67      0.36         3
        20.0       0.77      0.48      0.59        21
        22.0       0.62      0.74      0.67        80
        23.0       0.60      0.65      0.62        84
        24.0       0.74      0.77      0.75       243
        25.0       0.79      0.76      0.77        70
        26.0       0.76      0.74      0.75       112
        27.0       0.75      0.73      0.74        41
        28.0       0.75      0.49      0.60        79
        29.0       0.00      0.00      0.00         1

    accuracy                           0.73      1042
   macro avg       0.63      0.64      0.62      104

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:

#Dimenionality reduction. Only using the 100 best features er category

lsa = TruncatedSVD(n_components=100, 
                   n_iter=10, 
                   random_state=3)

X = lsa.fit_transform(df.description)
X.shape

In [44]:
df['description'][1]

'thanks watching happy holidays everyone links socials belownnnsocials maddiezieglern maddiestylennmaddie style website https wwwmaddiestylecomnnthis holiday sweater https wwwmaddiestylecom thisismyholidaysweaternnxoxoooo'

In [48]:
print_plot(1)

thanks watching happy holidays everyone links socials belownnnsocials maddiezieglern maddiestylennmaddie style website https wwwmaddiestylecomnnthis holiday sweater https wwwmaddiestylecom thisismyholidaysweaternnxoxoooo
Tag: 24.0


In [60]:
print_plot(1)

thanks watching happy holidays everyone links socials belownnnsocials maddiezieglern maddiestylennmaddie style website https wwwmaddiestylecomnnthis holiday sweater https wwwmaddiestylecom thisismyholidaysweaternnxoxoooo
Tag: 24.0


In [125]:
df1 = pd.read_csv('train.csv')
# def print_plot11(index):
#     example = df1[df1.index == index][['description', 'category_id']].values[0]
#     if len(example) > 0:
#         print(example[0])
#         print('Tag:', example[1])

# print_plot11(10)

In [153]:
print_plot11(1)

thanks for watching!!! happy holidays everyone!! links and socials below!\n\n\nsocials @maddieziegler\n@maddiestyle\n\nmaddie style website - https://www.maddiestyle.com\n\nthis is my holiday sweater - https://www.maddiestyle.com/this-is-my-holiday-sweater\n\nxoxoooo
Tag: 24.0


In [152]:
df1 = pd.read_csv('train.csv')
clean_text(df1.description[1])

'thanks watching happy holidays everyone links socials nsocials maddieziegler maddiestyle nmaddie style website maddiestyle nthis holiday sweater maddiestyle holiday sweater nxoxoooo'

In [148]:
df1.description[1]

'thanks for watching!!! happy holidays everyone!! links and socials below!\\n\\n\\nsocials @maddieziegler\\n@maddiestyle\\n\\nmaddie style website - https://www.maddiestyle.com\\n\\nthis is my holiday sweater - https://www.maddiestyle.com/this-is-my-holiday-sweater\\n\\nxoxoooo'

In [14]:

'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''Display'''
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings

In [10]:
texts = df['description'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X = tfidf_vectorizer.fit_transform(texts) #features
y = df['category_id'].values #target
X=X.toarray()

In [11]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#Creating a dict of the models
model_dict = {#'Dummy' : DummyClassifier(random_state=3),
              'Stochastic Gradient Descent' : SGDClassifier(random_state=3, loss='log'),
              'Random Forest': RandomForestClassifier(random_state=3),
              'Decsision Tree': DecisionTreeClassifier(random_state=3),
#               'AdaBoost': AdaBoostClassifier(random_state=3),
#               'Gaussian Naive Bayes': GaussianNB(),
              'K Nearest Neighbor': KNeighborsClassifier()}

# Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True, 
                                                    stratify = y, 
                                                    random_state = 3)

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in model_dict.items():   
        model_name.append(k)
        v.fit(X_train, y_train)
        y_pred = v.predict(X_test)
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

model_score_df(model_dict)

/home/himanshu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,.;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))
# main=set(['http','https','nhttp','n','com','www'])
# STOPWORDS=STOPWORDS|main

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
#     text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(' ', text) # delete symbols which are in BAD_SYMBOLS_RE from text
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
# df['description'] = df['description'].apply(clean_text)
# print_plot(1)

In [16]:
txt="Революция 1917 года и последовавшие за ней политические, социальные и экономические изменения принесли новую терминологию и значительно увеличили количество международных слов в русском словаре. Реформа орфографии 1918 года придала письменному русскому языку современный вид. Грамотность стала почти всеобщей. Достижения в военной, научной, технологической областях. области искусства, а также освоение космоса придали современной России всемирный престиж, который шел вместе с ее имиджем сверхдержавы."

In [17]:
txt=clean_text(txt)

In [18]:
txt

'          1917                                                                                                                                                                                                        1918                                                                                                                                                                                                                                                                                            '